In [13]:
import sys
print(sys.path)


!pip install fasterrisk

['/home/msr216/.conda/envs/fasterrisk/lib/python39.zip', '/home/msr216/.conda/envs/fasterrisk/lib/python3.9', '/home/msr216/.conda/envs/fasterrisk/lib/python3.9/lib-dynload', '', '/home/msr216/.conda/envs/fasterrisk/lib/python3.9/site-packages']


In [14]:
import numpy as np
import pandas as pd
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, mean_squared_error, confusion_matrix, accuracy_score
from matplotlib import pyplot as plt
from sklearn.utils import resample

In [3]:
df = pd.read_csv("diabetes_train.csv")
unique_count = df.iloc[:, -1].nunique()
print(f"Number of unique values in the last column: {unique_count}")

df = pd.read_csv("diabetes_test.csv")
unique_count = df.iloc[:, -1].nunique()
print(f"Number of unique values in the last column: {unique_count}")

Number of unique values in the last column: 2
Number of unique values in the last column: 2


In [4]:
df = pd.read_csv("diabetes_train.csv")
output = df['Outcome_1']
df = df.iloc[:,:-1]
df= pd.get_dummies(df,columns = df.columns)
print(df)
df = pd.concat([df,output],axis=1)
df.replace(df.iloc[:,-1].unique()[0],1,inplace=True)
df.replace(df.iloc[:,-1].unique()[[0]],-1,inplace=True)
h = df.columns
print(df)

     Glucose_0  Glucose_1  BloodPressure_0  BloodPressure_1  SkinThickness_0  \
0            1          0                1                0                1   
1            1          0                0                1                0   
2            0          1                1                0                1   
3            0          1                1                0                1   
4            0          1                0                1                0   
..         ...        ...              ...              ...              ...   
609          0          1                1                0                0   
610          1          0                0                1                1   
611          1          0                0                1                0   
612          0          1                1                0                1   
613          0          1                0                1                1   

     SkinThickness_1  Insulin_0  Insuli

In [5]:
import pandas as pd

# Load the test dataset
df2 = pd.read_csv("diabetes_test.csv")
output = df2['Outcome_1']  # Store the output column
df2 = df2.iloc[:, :-1]  # Drop the output column from features

# One-hot encoding for the test data
df2 = pd.get_dummies(df2, columns=df2.columns)
print("Processed Test DataFrame (before concatenation):")
print(df2)

# Add the output column back to the test dataframe
df2['Outcome_1'] = output

# Replace output values with 1 and -1
unique_test = df2['Outcome_1'].unique()
if len(unique_test) > 1:
    df2.replace(unique_test[0], 1, inplace=True)
    df2.replace(unique_test[1], -1, inplace=True)
elif len(unique_test) == 1:
    df2['output'] = 1 if unique_test[0] == 1 else -1  # Adjust as needed

# Ensure the test DataFrame has the same columns as the training DataFrame
# Assuming 'h' is defined from the training DataFrame earlier
df2 = df2.reindex(columns=h, fill_value=0)

# Final output
print("Final Processed Test DataFrame:")
print(df2)


Processed Test DataFrame (before concatenation):
     Glucose_0  Glucose_1  BloodPressure_0  BloodPressure_1  SkinThickness_0  \
0            1          0                1                0                0   
1            1          0                0                1                0   
2            1          0                1                0                1   
3            1          0                0                1                1   
4            0          1                0                1                1   
..         ...        ...              ...              ...              ...   
149          0          1                0                1                1   
150          1          0                1                0                0   
151          1          0                0                1                1   
152          0          1                0                1                0   
153          1          0                0                1            

In [6]:
import pandas as pd

# Load the training dataset
df = pd.read_csv("diabetes_train.csv")
output = df['Outcome_1']
df = df.iloc[:, :-1]  # Drop the output column from features

# Print the original data (already one-hot encoded)
print("Processed Training DataFrame:")
print(df)

# Add the output column back to the training DataFrame
df['Outcome_1'] = output

# Replace output values (fix the column name to 'Outcome_1' here)
df['Outcome_1'].replace(0, -1, inplace=True)
df['Outcome_1'].replace(1, 1, inplace=True)

# Store the column names for later use
h = df.columns

# Load the test dataset
df2 = pd.read_csv("diabetes_test.csv")
output2 = df2['Outcome_1']
df2 = df2.iloc[:, :-1]  # Drop the output column from features

# Print the original test data (already one-hot encoded)
print("Processed Test DataFrame (before concatenation):")
print(df2)

# Add the output column back to the test DataFrame
df2['Outcome_1'] = output2

# Replace output values (fix the column name to 'Outcome_1' here)
df2['Outcome_1'].replace(0, -1, inplace=True)
df2['Outcome_1'].replace(1, 1, inplace=True)

# Ensure the test DataFrame has the same columns as the training DataFrame
df2 = df2.reindex(columns=h, fill_value=0)

# Final output
print("Final Processed Test DataFrame:")
print(df2)



Processed Training DataFrame:
     Glucose  BloodPressure  SkinThickness  Insulin  BMI  \
0          0              0              0        0    0   
1          0              1              1        0    0   
2          1              0              0        1    0   
3          1              0              0        0    0   
4          1              1              1        1    1   
..       ...            ...            ...      ...  ...   
609        1              0              1        1    0   
610        0              1              0        0    0   
611        0              1              1        0    1   
612        1              0              0        0    1   
613        1              1              0        0    0   

     DiabetesPedigreeFunction  Age  hypertension  heart_disease  HbA1c_level  \
0                           0    0             0              0            1   
1                           1    1             0              0            1   
2        

In [7]:
X_train = df.iloc[:,:-1].to_numpy()
y_train = df['Outcome_1'].to_numpy()

X_test = df2.iloc[:,:-1].to_numpy()
y_test = df2['Outcome_1'].to_numpy()

In [8]:
import time
from fasterrisk.fasterrisk import RiskScoreOptimizer, RiskScoreClassifier
from sklearn.metrics import precision_score, recall_score, roc_auc_score

# Start the total runtime timer
start_time = time.time()

sparsity = 10  # produce a risk score model with 10 nonzero coefficients

# Initialize a risk score optimizer
m = RiskScoreOptimizer(X=X_train, y=y_train, k=sparsity)

# Perform optimization
m.optimize()

# Get all top m solutions from the final diverse pool
arr_multiplier, arr_intercept, arr_coefficients = m.get_models()

# Get the first solution from the final diverse pool
multiplier, intercept, coefficients = m.get_models(model_index=0)

# Feature names
X_featureNames = df.iloc[:, :-1].columns.values.tolist()

# Create a classifier
clf = RiskScoreClassifier(multiplier=multiplier, intercept=intercept, coefficients=coefficients, featureNames=X_featureNames)

# Get the predicted label
y_pred = clf.predict(X=X_train)

# Get the probability of predicting y[i] with label +1
y_pred_prob = clf.predict_prob(X=X_train)

# Compute the logistic loss
logisticLoss_train = clf.compute_logisticLoss(X=X_train, y=y_train)

# Get accuracy and area under the ROC curve (AUC)
acc_train, auc_train = clf.get_acc_and_auc(X=X_train, y=y_train)

# Calculate precision and recall
precision_train = precision_score(y_train, y_pred, average='weighted')
recall_train = recall_score(y_train, y_pred, average='weighted')

# End the total runtime timer
end_time = time.time()
total_run_time = end_time - start_time

# Print the results
print("Accuracy: ", acc_train)
print("AUC: ", auc_train)
print("Precision: ", precision_train)
print("Recall: ", recall_train)

# Print the total runtime
print(f"Total Run Time: {total_run_time:.4f} seconds")

# Optionally, you can print the risk score model information:
print("Risk Score Model Coefficients:")
print("Multiplier: ", multiplier)
print("Intercept: ", intercept)
print("Coefficients: ", coefficients)


Accuracy:  0.744299674267101
AUC:  0.7951834029948602
Precision:  0.7363658257280751
Recall:  0.744299674267101
Total Run Time: 4.0259 seconds
Risk Score Model Coefficients:
Multiplier:  1.0
Intercept:  -3.0
Coefficients:  [ 2.  0.  0.  0.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0. -1.  5.  0.  0.]


In [9]:
from fasterrisk.fasterrisk import RiskScoreOptimizer, RiskScoreClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix

# Sparsity: Number of non-zero coefficients in the model
sparsity = 10  # Produce a risk score model with 10 nonzero coefficients

# Initialize a risk score optimizer with training data
m = RiskScoreOptimizer(X=X_train, y=y_train, k=sparsity)

# Perform optimization
m.optimize()

# Get all top m solutions from the final diverse pool
arr_multiplier, arr_intercept, arr_coefficients = m.get_models()

# Get the first solution from the final diverse pool
multiplier, intercept, coefficients = m.get_models(model_index=0)

# Feature names
X_featureNames = df2.iloc[:, :-1].columns.values.tolist()  # Assuming df2 has the same structure

# Create a classifier
clf = RiskScoreClassifier(multiplier=multiplier, intercept=intercept, coefficients=coefficients, featureNames=X_featureNames)

# Get the predicted label for the test set
y_test_pred = clf.predict(X=X_test)

# Get the probability of predicting y[i] with label +1 for the test set
try:
    y_test_pred_prob = clf.predict_prob(X=X_test)
except Exception as e:
    print("Error while predicting probabilities:", e)
    y_test_pred_prob = (intercept + X_test.dot(coefficients)) / multiplier

# Compute the logistic loss for the test set
logisticLoss_test = clf.compute_logisticLoss(X=X_test, y=y_test)

# Get accuracy and area under the ROC curve (AUC) for the test set
acc_test, auc_test = clf.get_acc_and_auc(X=X_test, y=y_test)

# Calculate precision and recall for the test set
precision_test = precision_score(y_test, y_test_pred, average='weighted')
recall_test = recall_score(y_test, y_test_pred, average='weighted')

# Print the results for test metrics
print("Test Metrics:")
print("Accuracy: ", acc_test)
print("AUC: ", auc_test)
print("Precision: ", precision_test)
print("Recall: ", recall_test)


print("\nTesting Metrics:")
print("Accuracy: ", acc_test)
print("AUC: ", auc_test)
print("Precision: ", precision_test)
print("Recall: ", recall_test)

# Calculate confusion matrix for the test set
cm_test = confusion_matrix(y_test, y_test_pred)
cm_train = confusion_matrix(y_train, y_train_pred)

# Print the confusion matrix
print("\nConfusion Matrix (Train):")
print(cm_train)

print("\nConfusion Matrix (Test):")
print(cm_test)

# Print the risk score model card
clf.print_model_card()



Test Metrics:
Accuracy:  0.6623376623376623
AUC:  0.7455463728191001
Precision:  0.7085330776605945
Recall:  0.6623376623376623

Testing Metrics:
Accuracy:  0.6623376623376623
AUC:  0.7455463728191001
Precision:  0.7085330776605945
Recall:  0.6623376623376623


NameError: name 'y_train_pred' is not defined

In [12]:
print("Accuracy: ", acc_train)
print("AUC: ", auc_train)
print("Precision: ", precision_train)
print("Recall: ", recall_train)


Accuracy:  0.6623376623376623
AUC:  0.7455463728191001
Precision:  0.7363658257280751
Recall:  0.744299674267101


In [11]:
acc_train, auc_train = clf.get_acc_and_auc(X = X_test, y = y_test)
print("Accuracy: " , acc_train, "\nAUC: ",auc_train)

Accuracy:  0.6623376623376623 
AUC:  0.7455463728191001


In [47]:
logisticLoss_test = clf.compute_logisticLoss(X = X_test, y = y_test)

print("Training Loss: ",logisticLoss_train)
print("Test Loss: ",logisticLoss_test)

Training Loss:  318.05912905806065
Test Loss:  85.49035484006396


Processed Test DataFrame (before concatenation):
     Glucose_0  Glucose_1  BloodPressure_0  BloodPressure_1  SkinThickness_0  \
0            1          0                1                0                0   
1            1          0                0                1                0   
2            1          0                1                0                1   
3            1          0                0                1                1   
4            0          1                0                1                1   
..         ...        ...              ...              ...              ...   
149          0          1                0                1                1   
150          1          0                1                0                0   
151          1          0                0                1                1   
152          0          1                0                1                0   
153          1          0                0                1            

/home/msr216/.conda/envs/fasterrisk/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [54]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score
from fasterrisk.fasterrisk import RiskScoreOptimizer, RiskScoreClassifier

# Load the test dataset
df = pd.read_csv("diabetes_test.csv")
primary = df['Outcome_1']  # assuming 'Outcome_1' is the target variable
df = df.iloc[:, :-1]  # drop the target variable column
df = pd.get_dummies(df, columns=df.columns)  # one-hot encoding of features
df = pd.concat([df, primary], axis=1)

# Replace target values
df.replace(df.iloc[:, -1].unique()[1], 1, inplace=True)  # assuming 1 is the positive class
df.replace(df.iloc[:, -1].unique()[0], -1, inplace=True)  # assuming 0 is the negative class

# Load the validation/test set
df2 = pd.read_csv("diabetes_test.csv")
primary2 = df2['Outcome_1']  # same target variable as in df
df2 = df2.iloc[:, :-1]  # drop the target variable column
df2 = pd.get_dummies(df2, columns=df2.columns)
df2 = pd.concat([df2, primary2], axis=1)

# Replace target values in test set
df2.replace(df2.iloc[:, -1].unique()[1], 1, inplace=True)  # assuming 1 is the positive class
df2.replace(df2.iloc[:, -1].unique()[0], -1, inplace=True)  # assuming 0 is the negative class

# Handle missing values (if necessary)
empty = pd.DataFrame(columns=df.columns)  # assuming columns 'h' exist
df2 = pd.concat([empty, df2], axis=0, join='outer')
df2 = df2[df2.columns.intersection(df.columns)]
df2.fillna(0, inplace=True)

# Prepare features and target variables for training and testing
X_train = df.iloc[:, :-1].to_numpy()
y_train = df['Outcome_1'].to_numpy()
X_test = df2.iloc[:, :-1].to_numpy()
y_test = df2['Outcome_1'].to_numpy()

# Sparsity setting
sparsity = 10  # produce a risk score model with 5 nonzero coefficients

# Initialize and optimize the model
m = RiskScoreOptimizer(X=X_train, y=y_train, k=sparsity)
m.optimize()

# Get the first solution from the optimization pool
multiplier, intercept, coefficients = m.get_models(model_index=0)
X_featureNames = df.iloc[:, :-1].columns.values.tolist()

# Create classifier
clf = RiskScoreClassifier(multiplier=multiplier, intercept=intercept, coefficients=coefficients, featureNames=X_featureNames)

# Predictions for training data (not needed for metrics here, but useful for model insights)
y_pred_train = clf.predict(X=X_train)
y_pred_prob_train = clf.predict_prob(X=X_train)

# Compute training loss (if needed for insights)
logisticLoss_train = clf.compute_logisticLoss(X=X_train, y=y_train)

# Predictions for test data
y_pred_test = clf.predict(X=X_test)
y_pred_prob_test = clf.predict_prob(X=X_test)

# Compute test loss (optional)
logisticLoss_test = clf.compute_logisticLoss(X=X_test, y=y_test)

# Print model evaluation
print("Test Loss: ", logisticLoss_test)

# Compute accuracy, precision, and recall on test set
accuracy = accuracy_score(y_test, y_pred_test)
precision = precision_score(y_test, y_pred_test, average='binary', pos_label=1)
recall = recall_score(y_test, y_pred_test, average='binary', pos_label=1)

print("Accuracy on Test Set: ", accuracy)
print("Precision on Test Set: ", precision)
print("Recall on Test Set: ", recall)


TypeError: loop of ufunc does not support argument 0 of type float which has no callable exp method

In [12]:
df = pd.read_csv("cup.CSV")
primary = df['Primary']
df = df.iloc[:,:-1]
df= pd.get_dummies(df,columns = df.columns)
df = pd.concat([df,primary],axis=1)
df.replace(df.iloc[:,-1].unique()[3],1,inplace=True)
df.replace(df.iloc[:,-1].unique()[[0,1,2,4,5,6,7,8,9]],-1,inplace=True)

df2 = pd.read_csv("cup2.csv")
primary2 = df2['Primary']
df2 = df2.iloc[:,:-1]
df2= pd.get_dummies(df2,columns = df2.columns)
df2 = pd.concat([df2,primary2],axis=1)
df2.replace(df2.iloc[:,-1].unique()[1],1,inplace=True)
df2.replace(df2.iloc[:,-1].unique()[[0,2,3,4,5,6,7,8,9]],-1,inplace=True)
empty = pd.DataFrame(columns = h)
df2 = pd.concat([empty, df2], axis=0, join='outer')
df2 = df2[df2.columns.intersection(h)]
df2.fillna(0, inplace=True)

X_train = df.iloc[:,:-1].to_numpy()
y_train = df['Primary'].to_numpy()

X_test = df2.iloc[:,:-1].to_numpy()
y_test = df2['Primary'].to_numpy()

from fasterrisk.fasterrisk import RiskScoreOptimizer, RiskScoreClassifier

sparsity = 10 # produce a risk score model with 5 nonzero coefficients

# initialize a risk score optimizer
m = RiskScoreOptimizer(X = X_train, y = y_train, k = sparsity)

# perform optimization
m.optimize()

# get all top m solutions from the final diverse pool
arr_multiplier, arr_intercept, arr_coefficients = m.get_models() # get m solutions from the diverse pool; Specifically, arr_multiplier.shape=(m, ), arr_intercept.shape=(m, ), arr_coefficients.shape=(m, p)

# get the first solution from the final diverse pool by passing an optional model_index; models are ranked in order of increasing logistic loss
multiplier, intercept, coefficients = m.get_models(model_index = 0) # get the first solution (smallest logistic loss) from the diverse pool; Specifically, multiplier.shape=(1, ), intercept.shape=(1, ), coefficients.shape=(p, )
X_featureNames = df.iloc[:,:-1].columns.values.tolist()# X_featureNames is a list of strings, each of which is the feature name

# create a classifier
clf = RiskScoreClassifier(multiplier = multiplier, intercept = intercept, coefficients = coefficients, featureNames = X_featureNames)

# get the predicted label
y_pred = clf.predict(X = X_train)

# get the probability of predicting y[i] with label +1
y_pred_prob = clf.predict_prob(X = X_train)

# compute the logistic loss
logisticLoss_train = clf.compute_logisticLoss(X = X_train, y = y_train)
logisticLoss_test = clf.compute_logisticLoss(X = X_test, y = y_test)
# print the risk score model card
clf.print_model_card()

# get accuracy and area under the ROC curve (AUC)
print("Training Loss: ",logisticLoss_train)
print("Test Loss: ",logisticLoss_test)

acc_train, auc_train = clf.get_acc_and_auc(X = X_train, y = y_train)
print("Accuracy: " , acc_train, "\nAUC: ",auc_train)

acc_train, auc_train = clf.get_acc_and_auc(X = X_test, y = y_test)
print("Accuracy: " , acc_train, "\nAUC: ",auc_train)

The Risk Score is:
1.    RBFOX1_-1     -3 point(s) |   ...
2.     RBFOX1_1      3 point(s) | + ...
3.    IL17RE_-1     -4 point(s) | + ...
4.     GATA3_-1     -2 point(s) | + ...
5.       IHO1_0      2 point(s) | + ...
6.    MIR4732_2     -5 point(s) | + ...
7.      LCOR_-1      4 point(s) | + ...
8.     PTF1A_-1      2 point(s) | + ...
9.      ITGB1_2      5 point(s) | + ...
10.    MEGF10_-1      2 point(s) | + ...
                          SCORE | =    
SCORE |  -14.0  |  -12.0  |  -11.0  |  -10.0  |  -9.0  |  -8.0  |  -7.0  |  -6.0  |  -5.0  |  -4.0  |  -3.0  |  -2.0  |  -1.0  |   0.0  |   1.0  |   2.0  |
RISK  |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.1% |   0.1% |   0.2% |   0.3% |   0.4% |   0.7% |   1.1% |   1.7% |   2.6% |   4.0% |   6.2% |
SCORE |   3.0  |   4.0  |   5.0  |   6.0  |   7.0  |   8.0  |   9.0  |  10.0  |  11.0  |  12.0  |  13.0  |  14.0  |  15.0  |  16.0  |  18.0  |
RISK  |   9.4% |  14.0% |  20.4% |  28.8% |  38.8% |  50.0% |  61.2% |  71.2% |  79.6% | 

In [13]:
df = pd.read_csv("cup.CSV")
primary = df['Primary']
df = df.iloc[:,:-1]
df= pd.get_dummies(df,columns = df.columns)
df = pd.concat([df,primary],axis=1)
df.replace(df.iloc[:,-1].unique()[4],1,inplace=True)
df.replace(df.iloc[:,-1].unique()[[0,1,2,3,5,6,7,8,9]],-1,inplace=True)

df2 = pd.read_csv("cup2.csv")
primary2 = df2['Primary']
df2 = df2.iloc[:,:-1]
df2= pd.get_dummies(df2,columns = df2.columns)
df2 = pd.concat([df2,primary2],axis=1)
df2.replace(df2.iloc[:,-1].unique()[2],1,inplace=True)
df2.replace(df2.iloc[:,-1].unique()[[0,1,3,4,5,6,7,8,9]],-1,inplace=True)
empty = pd.DataFrame(columns = h)
df2 = pd.concat([empty, df2], axis=0, join='outer')
df2 = df2[df2.columns.intersection(h)]
df2.fillna(0, inplace=True)

X_train = df.iloc[:,:-1].to_numpy()
y_train = df['Primary'].to_numpy()

X_test = df2.iloc[:,:-1].to_numpy()
y_test = df2['Primary'].to_numpy()

from fasterrisk.fasterrisk import RiskScoreOptimizer, RiskScoreClassifier

sparsity = 10 # produce a risk score model with 5 nonzero coefficients

# initialize a risk score optimizer
m = RiskScoreOptimizer(X = X_train, y = y_train, k = sparsity)

# perform optimization
m.optimize()

# get all top m solutions from the final diverse pool
arr_multiplier, arr_intercept, arr_coefficients = m.get_models() # get m solutions from the diverse pool; Specifically, arr_multiplier.shape=(m, ), arr_intercept.shape=(m, ), arr_coefficients.shape=(m, p)

# get the first solution from the final diverse pool by passing an optional model_index; models are ranked in order of increasing logistic loss
multiplier, intercept, coefficients = m.get_models(model_index = 0) # get the first solution (smallest logistic loss) from the diverse pool; Specifically, multiplier.shape=(1, ), intercept.shape=(1, ), coefficients.shape=(p, )
X_featureNames = df.iloc[:,:-1].columns.values.tolist()# X_featureNames is a list of strings, each of which is the feature name

# create a classifier
clf = RiskScoreClassifier(multiplier = multiplier, intercept = intercept, coefficients = coefficients, featureNames = X_featureNames)

# get the predicted label
y_pred = clf.predict(X = X_train)

# get the probability of predicting y[i] with label +1
y_pred_prob = clf.predict_prob(X = X_train)

# compute the logistic loss
logisticLoss_train = clf.compute_logisticLoss(X = X_train, y = y_train)
logisticLoss_test = clf.compute_logisticLoss(X = X_test, y = y_test)
# print the risk score model card
clf.print_model_card()

# get accuracy and area under the ROC curve (AUC)
print("Training Loss: ",logisticLoss_train)
print("Test Loss: ",logisticLoss_test)

acc_train, auc_train = clf.get_acc_and_auc(X = X_train, y = y_train)
print("Accuracy: " , acc_train, "\nAUC: ",auc_train)

acc_train, auc_train = clf.get_acc_and_auc(X = X_test, y = y_test)
print("Accuracy: " , acc_train, "\nAUC: ",auc_train)

The Risk Score is:
1.    NKIRAS1_0      5 point(s) |   ...
2. CDKN2A-DT_-2      3 point(s) | + ...
3.   MIR378C_-1      4 point(s) | + ...
4.    ERICH6_-1      3 point(s) | + ...
5.       SIL1_0      2 point(s) | + ...
6.      NICN1_1      5 point(s) | + ...
7.     EDEM1_-1     -3 point(s) | + ...
8.      APOD_-1      3 point(s) | + ...
9.     PLXDC1_1     -5 point(s) | + ...
10.         SI_1      3 point(s) | + ...
                          SCORE | =    
SCORE |  -8.0  |  -6.0  |  -5.0  |  -4.0  |  -3.0  |  -2.0  |  -1.0  |   0.0  |   1.0  |   2.0  |   3.0  |   4.0  |   5.0  |   6.0  |   7.0  |   8.0  |   9.0  |  10.0  |
RISK  |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.1% |   0.1% |   0.3% |   0.5% |   1.0% |   2.0% |   3.7% |   6.9% |  12.4% |
SCORE |  11.0  |  12.0  |  13.0  |  14.0  |  15.0  |  16.0  |  17.0  |  18.0  |  19.0  |  20.0  |  21.0  |  22.0  |  23.0  |  24.0  |  25.0  |  26.0  |  28.0  |
RISK  |  21.4% |  34.3% |  50.0% |  65.

In [14]:
df = pd.read_csv("cup.CSV")
primary = df['Primary']
df = df.iloc[:,:-1]
df= pd.get_dummies(df,columns = df.columns)
df = pd.concat([df,primary],axis=1)
df.replace(df.iloc[:,-1].unique()[5],1,inplace=True)
df.replace(df.iloc[:,-1].unique()[[0,1,2,3,4,6,7,8,9]],-1,inplace=True)

df2 = pd.read_csv("cup2.csv")
primary2 = df2['Primary']
df2 = df2.iloc[:,:-1]
df2= pd.get_dummies(df2,columns = df2.columns)
df2 = pd.concat([df2,primary2],axis=1)
df2.replace(df2.iloc[:,-1].unique()[6],1,inplace=True)
df2.replace(df2.iloc[:,-1].unique()[[0,1,2,3,4,5,7,8,9]],-1,inplace=True)
empty = pd.DataFrame(columns = h)
df2 = pd.concat([empty, df2], axis=0, join='outer')
df2 = df2[df2.columns.intersection(h)]
df2.fillna(0, inplace=True)

X_train = df.iloc[:,:-1].to_numpy()
y_train = df['Primary'].to_numpy()

X_test = df2.iloc[:,:-1].to_numpy()
y_test = df2['Primary'].to_numpy()

from fasterrisk.fasterrisk import RiskScoreOptimizer, RiskScoreClassifier

sparsity = 10 # produce a risk score model with 5 nonzero coefficients

# initialize a risk score optimizer
m = RiskScoreOptimizer(X = X_train, y = y_train, k = sparsity)

# perform optimization
m.optimize()

# get all top m solutions from the final diverse pool
arr_multiplier, arr_intercept, arr_coefficients = m.get_models() # get m solutions from the diverse pool; Specifically, arr_multiplier.shape=(m, ), arr_intercept.shape=(m, ), arr_coefficients.shape=(m, p)

# get the first solution from the final diverse pool by passing an optional model_index; models are ranked in order of increasing logistic loss
multiplier, intercept, coefficients = m.get_models(model_index = 0) # get the first solution (smallest logistic loss) from the diverse pool; Specifically, multiplier.shape=(1, ), intercept.shape=(1, ), coefficients.shape=(p, )
X_featureNames = df.iloc[:,:-1].columns.values.tolist()# X_featureNames is a list of strings, each of which is the feature name

# create a classifier
clf = RiskScoreClassifier(multiplier = multiplier, intercept = intercept, coefficients = coefficients, featureNames = X_featureNames)

# get the predicted label
y_pred = clf.predict(X = X_train)

# get the probability of predicting y[i] with label +1
y_pred_prob = clf.predict_prob(X = X_train)

# compute the logistic loss
logisticLoss_train = clf.compute_logisticLoss(X = X_train, y = y_train)
logisticLoss_test = clf.compute_logisticLoss(X = X_test, y = y_test)
# print the risk score model card
clf.print_model_card()

# get accuracy and area under the ROC curve (AUC)
print("Training Loss: ",logisticLoss_train)
print("Test Loss: ",logisticLoss_test)

acc_train, auc_train = clf.get_acc_and_auc(X = X_train, y = y_train)
print("Accuracy: " , acc_train, "\nAUC: ",auc_train)

acc_train, auc_train = clf.get_acc_and_auc(X = X_test, y = y_test)
print("Accuracy: " , acc_train, "\nAUC: ",auc_train)

The Risk Score is:
1.    YTHDC2_-1      2 point(s) |   ...
2.     NEK10_-1      2 point(s) | + ...
3.     SRCIN1_2     -5 point(s) | + ...
4.     METTL6_1     -4 point(s) | + ...
5.       MBL2_1      2 point(s) | + ...
6.    PCDHB12_2      4 point(s) | + ...
7.    POU4F3_-1      1 point(s) | + ...
8.   PCDHGA5_-2      5 point(s) | + ...
9.   SNORD69_-1      1 point(s) | + ...
10.     ACSL5_-1     -2 point(s) | + ...
                          SCORE | =    
SCORE |  -11.0  |  -10.0  |  -9.0  |  -8.0  |  -7.0  |  -6.0  |  -5.0  |  -4.0  |  -3.0  |  -2.0  |  -1.0  |   0.0  |   1.0  |   2.0  |   3.0  |
RISK  |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.1% |   0.2% |   0.3% |   0.5% |   0.9% |   1.6% |   2.8% |   5.0% |
SCORE |   4.0  |   5.0  |   6.0  |   7.0  |   8.0  |   9.0  |  10.0  |  11.0  |  12.0  |  13.0  |  14.0  |  15.0  |  16.0  |  17.0  |
RISK  |   8.6% |  14.6% |  23.5% |  35.7% |  50.0% |  64.3% |  76.5% |  85.4% |  91.4% |  95.0% |  97.2% |  98.4% |  9

In [15]:
df = pd.read_csv("cup.CSV")
primary = df['Primary']
df = df.iloc[:,:-1]
df= pd.get_dummies(df,columns = df.columns)
df = pd.concat([df,primary],axis=1)
df.replace(df.iloc[:,-1].unique()[6],1,inplace=True)
df.replace(df.iloc[:,-1].unique()[[0,1,2,3,4,5,7,8,9]],-1,inplace=True)

df2 = pd.read_csv("cup2.csv")
primary2 = df2['Primary']
df2 = df2.iloc[:,:-1]
df2= pd.get_dummies(df2,columns = df2.columns)
df2 = pd.concat([df2,primary2],axis=1)
df2.replace(df2.iloc[:,-1].unique()[7],1,inplace=True)
df2.replace(df2.iloc[:,-1].unique()[[0,1,2,3,4,5,6,8,9]],-1,inplace=True)
empty = pd.DataFrame(columns = h)
df2 = pd.concat([empty, df2], axis=0, join='outer')
df2 = df2[df2.columns.intersection(h)]
df2.fillna(0, inplace=True)

X_train = df.iloc[:,:-1].to_numpy()
y_train = df['Primary'].to_numpy()

X_test = df2.iloc[:,:-1].to_numpy()
y_test = df2['Primary'].to_numpy()

from fasterrisk.fasterrisk import RiskScoreOptimizer, RiskScoreClassifier

sparsity = 10 # produce a risk score model with 5 nonzero coefficients

# initialize a risk score optimizer
m = RiskScoreOptimizer(X = X_train, y = y_train, k = sparsity)

# perform optimization
m.optimize()

# get all top m solutions from the final diverse pool
arr_multiplier, arr_intercept, arr_coefficients = m.get_models() # get m solutions from the diverse pool; Specifically, arr_multiplier.shape=(m, ), arr_intercept.shape=(m, ), arr_coefficients.shape=(m, p)

# get the first solution from the final diverse pool by passing an optional model_index; models are ranked in order of increasing logistic loss
multiplier, intercept, coefficients = m.get_models(model_index = 0) # get the first solution (smallest logistic loss) from the diverse pool; Specifically, multiplier.shape=(1, ), intercept.shape=(1, ), coefficients.shape=(p, )
X_featureNames = df.iloc[:,:-1].columns.values.tolist()# X_featureNames is a list of strings, each of which is the feature name

# create a classifier
clf = RiskScoreClassifier(multiplier = multiplier, intercept = intercept, coefficients = coefficients, featureNames = X_featureNames)

# get the predicted label
y_pred = clf.predict(X = X_train)

# get the probability of predicting y[i] with label +1
y_pred_prob = clf.predict_prob(X = X_train)

# compute the logistic loss
logisticLoss_train = clf.compute_logisticLoss(X = X_train, y = y_train)
logisticLoss_test = clf.compute_logisticLoss(X = X_test, y = y_test)
# print the risk score model card
clf.print_model_card()

# get accuracy and area under the ROC curve (AUC)
print("Training Loss: ",logisticLoss_train)
print("Test Loss: ",logisticLoss_test)

acc_train, auc_train = clf.get_acc_and_auc(X = X_train, y = y_train)
print("Accuracy: " , acc_train, "\nAUC: ",auc_train)

acc_train, auc_train = clf.get_acc_and_auc(X = X_test, y = y_test)
print("Accuracy: " , acc_train, "\nAUC: ",auc_train)

The Risk Score is:
1.     NLGN1_-1     -4 point(s) |   ...
2.    RHBDL3_-1      2 point(s) | + ...
3.     IQCF2_-1     -2 point(s) | + ...
4.    THNSL1_-1     -3 point(s) | + ...
5.       PURA_0     -3 point(s) | + ...
6.   RPL22L1_-2      5 point(s) | + ...
7.   RPL22L1_-1      3 point(s) | + ...
8.     PHF12_-1      3 point(s) | + ...
9.   SLC12A2_-1      3 point(s) | + ...
10.      CHSY3_0     -2 point(s) | + ...
                          SCORE | =    
SCORE |  -14.0  |  -12.0  |  -11.0  |  -10.0  |  -9.0  |  -8.0  |  -7.0  |  -6.0  |  -5.0  |  -4.0  |  -3.0  |  -2.0  |  -1.0  |   0.0  |   1.0  |
RISK  |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.1% |   0.1% |   0.2% |   0.4% |   0.7% |   1.3% |   2.4% |   4.3% |
SCORE |   2.0  |   3.0  |   4.0  |   5.0  |   6.0  |   7.0  |   8.0  |   9.0  |  10.0  |  11.0  |  12.0  |  13.0  |  14.0  |  16.0  |
RISK  |   7.8% |  13.5% |  22.5% |  35.0% |  50.0% |  65.0% |  77.5% |  86.5% |  92.2% |  95.7% |  97.6% |  98.7% | 

In [16]:
df = pd.read_csv("cup.CSV")
primary = df['Primary']
df = df.iloc[:,:-1]
df= pd.get_dummies(df,columns = df.columns)
df = pd.concat([df,primary],axis=1)
df.replace(df.iloc[:,-1].unique()[7],1,inplace=True)
df.replace(df.iloc[:,-1].unique()[[0,1,2,3,4,5,6,8,9]],-1,inplace=True)

df2 = pd.read_csv("cup2.csv")
primary2 = df2['Primary']
df2 = df2.iloc[:,:-1]
df2= pd.get_dummies(df2,columns = df2.columns)
df2 = pd.concat([df2,primary2],axis=1)
df2.replace(df2.iloc[:,-1].unique()[9],1,inplace=True)
df2.replace(df2.iloc[:,-1].unique()[[0,1,2,3,4,5,6,7,8]],-1,inplace=True)
empty = pd.DataFrame(columns = h)
df2 = pd.concat([empty, df2], axis=0, join='outer')
df2 = df2[df2.columns.intersection(h)]
df2.fillna(0, inplace=True)

X_train = df.iloc[:,:-1].to_numpy()
y_train = df['Primary'].to_numpy()

X_test = df2.iloc[:,:-1].to_numpy()
y_test = df2['Primary'].to_numpy()

from fasterrisk.fasterrisk import RiskScoreOptimizer, RiskScoreClassifier

sparsity = 10 # produce a risk score model with 5 nonzero coefficients

# initialize a risk score optimizer
m = RiskScoreOptimizer(X = X_train, y = y_train, k = sparsity)

# perform optimization
m.optimize()

# get all top m solutions from the final diverse pool
arr_multiplier, arr_intercept, arr_coefficients = m.get_models() # get m solutions from the diverse pool; Specifically, arr_multiplier.shape=(m, ), arr_intercept.shape=(m, ), arr_coefficients.shape=(m, p)

# get the first solution from the final diverse pool by passing an optional model_index; models are ranked in order of increasing logistic loss
multiplier, intercept, coefficients = m.get_models(model_index = 0) # get the first solution (smallest logistic loss) from the diverse pool; Specifically, multiplier.shape=(1, ), intercept.shape=(1, ), coefficients.shape=(p, )
X_featureNames = df.iloc[:,:-1].columns.values.tolist()# X_featureNames is a list of strings, each of which is the feature name

# create a classifier
clf = RiskScoreClassifier(multiplier = multiplier, intercept = intercept, coefficients = coefficients, featureNames = X_featureNames)

# get the predicted label
y_pred = clf.predict(X = X_train)

# get the probability of predicting y[i] with label +1
y_pred_prob = clf.predict_prob(X = X_train)

# compute the logistic loss
logisticLoss_train = clf.compute_logisticLoss(X = X_train, y = y_train)
logisticLoss_test = clf.compute_logisticLoss(X = X_test, y = y_test)
# print the risk score model card
clf.print_model_card()

# get accuracy and area under the ROC curve (AUC)
print("Training Loss: ",logisticLoss_train)
print("Test Loss: ",logisticLoss_test)

acc_train, auc_train = clf.get_acc_and_auc(X = X_train, y = y_train)
print("Accuracy: " , acc_train, "\nAUC: ",auc_train)

acc_train, auc_train = clf.get_acc_and_auc(X = X_test, y = y_test)
print("Accuracy: " , acc_train, "\nAUC: ",auc_train)

The Risk Score is:
1. CDKN2A-DT_-1     -1 point(s) |   ...
2.    RBFOX1_-2      4 point(s) | + ...
3.    CDKN2A_-2     -4 point(s) | + ...
4.    AKR1C6P_0      2 point(s) | + ...
5.      ALG3_-1     -2 point(s) | + ...
6.      NFAT5_1      3 point(s) | + ...
7.      NFAT5_2      5 point(s) | + ...
8.      LCOR_-1     -2 point(s) | + ...
9.     EDEM1_-1      2 point(s) | + ...
10.    PTPN23_-1     -1 point(s) | + ...
                          SCORE | =    
SCORE |  -10.0  |  -9.0  |  -8.0  |  -7.0  |  -6.0  |  -5.0  |  -4.0  |  -3.0  |  -2.0  |  -1.0  |   0.0  |   1.0  |   2.0  |   3.0  |
RISK  |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.1% |   0.2% |   0.7% |   1.8% |   4.7% |  11.9% |  26.9% |
SCORE |   4.0  |   5.0  |   6.0  |   7.0  |   8.0  |   9.0  |  10.0  |  11.0  |  12.0  |  13.0  |  14.0  |  15.0  |  16.0  |
RISK  |  50.0% |  73.1% |  88.1% |  95.3% |  98.2% |  99.3% |  99.8% |  99.9% | 100.0% | 100.0% | 100.0% | 100.0% | 100.0% |
Training Loss:  1081.

In [17]:
df = pd.read_csv("cup.CSV")
primary = df['Primary']
df = df.iloc[:,:-1]
df= pd.get_dummies(df,columns = df.columns)
df = pd.concat([df,primary],axis=1)
df.replace(df.iloc[:,-1].unique()[8],1,inplace=True)
df.replace(df.iloc[:,-1].unique()[[0,1,2,3,4,5,6,7,9]],-1,inplace=True)

df2 = pd.read_csv("cup2.csv")
primary2 = df2['Primary']
df2 = df2.iloc[:,:-1]
df2= pd.get_dummies(df2,columns = df2.columns)
df2 = pd.concat([df2,primary2],axis=1)
df2.replace(df2.iloc[:,-1].unique()[5],1,inplace=True)
df2.replace(df2.iloc[:,-1].unique()[[0,1,2,3,4,6,7,8,9]],-1,inplace=True)
empty = pd.DataFrame(columns = h)
df2 = pd.concat([empty, df2], axis=0, join='outer')
df2 = df2[df2.columns.intersection(h)]
df2.fillna(0, inplace=True)

X_train = df.iloc[:,:-1].to_numpy()
y_train = df['Primary'].to_numpy()

X_test = df2.iloc[:,:-1].to_numpy()
y_test = df2['Primary'].to_numpy()

from fasterrisk.fasterrisk import RiskScoreOptimizer, RiskScoreClassifier

sparsity = 10 # produce a risk score model with 5 nonzero coefficients

# initialize a risk score optimizer
m = RiskScoreOptimizer(X = X_train, y = y_train, k = sparsity)

# perform optimization
m.optimize()

# get all top m solutions from the final diverse pool
arr_multiplier, arr_intercept, arr_coefficients = m.get_models() # get m solutions from the diverse pool; Specifically, arr_multiplier.shape=(m, ), arr_intercept.shape=(m, ), arr_coefficients.shape=(m, p)

# get the first solution from the final diverse pool by passing an optional model_index; models are ranked in order of increasing logistic loss
multiplier, intercept, coefficients = m.get_models(model_index = 0) # get the first solution (smallest logistic loss) from the diverse pool; Specifically, multiplier.shape=(1, ), intercept.shape=(1, ), coefficients.shape=(p, )
X_featureNames = df.iloc[:,:-1].columns.values.tolist()# X_featureNames is a list of strings, each of which is the feature name

# create a classifier
clf = RiskScoreClassifier(multiplier = multiplier, intercept = intercept, coefficients = coefficients, featureNames = X_featureNames)

# get the predicted label
y_pred = clf.predict(X = X_train)

# get the probability of predicting y[i] with label +1
y_pred_prob = clf.predict_prob(X = X_train)

# compute the logistic loss
logisticLoss_train = clf.compute_logisticLoss(X = X_train, y = y_train)
logisticLoss_test = clf.compute_logisticLoss(X = X_test, y = y_test)
# print the risk score model card
clf.print_model_card()

# get accuracy and area under the ROC curve (AUC)
print("Training Loss: ",logisticLoss_train)
print("Test Loss: ",logisticLoss_test)

acc_train, auc_train = clf.get_acc_and_auc(X = X_train, y = y_train)
print("Accuracy: " , acc_train, "\nAUC: ",auc_train)

acc_train, auc_train = clf.get_acc_and_auc(X = X_test, y = y_test)
print("Accuracy: " , acc_train, "\nAUC: ",auc_train)

The Risk Score is:
1.       PIGW_1      2 point(s) |   ...
2.      AGTR1_1     -2 point(s) | + ...
3.     PRKCQ_-1     -4 point(s) | + ...
4.  B3GALNT1_-1      3 point(s) | + ...
5.      ALG3_-2      4 point(s) | + ...
6.      ALG3_-1      3 point(s) | + ...
7.       TNS4_2     -4 point(s) | + ...
8.     CHSY3_-1     -2 point(s) | + ...
9.        SI_-1      3 point(s) | + ...
10.     TPGS1_-1     -2 point(s) | + ...
                          SCORE | =    
SCORE |  -14.0  |  -12.0  |  -11.0  |  -10.0  |  -9.0  |  -8.0  |  -7.0  |  -6.0  |  -5.0  |  -4.0  |  -3.0  |  -2.0  |  -1.0  |   0.0  |
RISK  |   0.0% |   0.0% |   0.0% |   0.1% |   0.1% |   0.1% |   0.2% |   0.4% |   0.6% |   0.9% |   1.5% |   2.4% |   3.7% |   5.8% |
SCORE |   1.0  |   2.0  |   3.0  |   4.0  |   5.0  |   6.0  |   7.0  |   8.0  |   9.0  |  10.0  |  11.0  |  12.0  |  13.0  |  15.0  |
RISK  |   8.9% |  13.4% |  19.8% |  28.3% |  38.6% |  50.0% |  61.4% |  71.7% |  80.2% |  86.6% |  91.1% |  94.2% |  96.3% |  98.5% |


In [18]:
df = pd.read_csv("cup.CSV")
primary = df['Primary']
df = df.iloc[:,:-1]
df= pd.get_dummies(df,columns = df.columns)
df = pd.concat([df,primary],axis=1)
df.replace(df.iloc[:,-1].unique()[9],1,inplace=True)
df.replace(df.iloc[:,-1].unique()[[0,1,2,3,4,5,6,7,8]],-1,inplace=True)

df2 = pd.read_csv("cup2.csv")
primary2 = df2['Primary']
df2 = df2.iloc[:,:-1]
df2= pd.get_dummies(df2,columns = df2.columns)
df2 = pd.concat([df2,primary2],axis=1)
df2.replace(df2.iloc[:,-1].unique()[4],1,inplace=True)
df2.replace(df2.iloc[:,-1].unique()[[0,1,2,3,5,6,7,8,9]],-1,inplace=True)
empty = pd.DataFrame(columns = h)
df2 = pd.concat([empty, df2], axis=0, join='outer')
df2 = df2[df2.columns.intersection(h)]
df2.fillna(0, inplace=True)

X_train = df.iloc[:,:-1].to_numpy()
y_train = df['Primary'].to_numpy()

X_test = df2.iloc[:,:-1].to_numpy()
y_test = df2['Primary'].to_numpy()

from fasterrisk.fasterrisk import RiskScoreOptimizer, RiskScoreClassifier

sparsity = 10 # produce a risk score model with 5 nonzero coefficients

# initialize a risk score optimizer
m = RiskScoreOptimizer(X = X_train, y = y_train, k = sparsity)

# perform optimization
m.optimize()

# get all top m solutions from the final diverse pool
arr_multiplier, arr_intercept, arr_coefficients = m.get_models() # get m solutions from the diverse pool; Specifically, arr_multiplier.shape=(m, ), arr_intercept.shape=(m, ), arr_coefficients.shape=(m, p)

# get the first solution from the final diverse pool by passing an optional model_index; models are ranked in order of increasing logistic loss
multiplier, intercept, coefficients = m.get_models(model_index = 0) # get the first solution (smallest logistic loss) from the diverse pool; Specifically, multiplier.shape=(1, ), intercept.shape=(1, ), coefficients.shape=(p, )
X_featureNames = df.iloc[:,:-1].columns.values.tolist()# X_featureNames is a list of strings, each of which is the feature name

# create a classifier
clf = RiskScoreClassifier(multiplier = multiplier, intercept = intercept, coefficients = coefficients, featureNames = X_featureNames)

# get the predicted label
y_pred = clf.predict(X = X_train)

# get the probability of predicting y[i] with label +1
y_pred_prob = clf.predict_prob(X = X_train)

# compute the logistic loss
logisticLoss_train = clf.compute_logisticLoss(X = X_train, y = y_train)
logisticLoss_test = clf.compute_logisticLoss(X = X_test, y = y_test)
# print the risk score model card
clf.print_model_card()

# get accuracy and area under the ROC curve (AUC)
print("Training Loss: ",logisticLoss_train)
print("Test Loss: ",logisticLoss_test)

acc_train, auc_train = clf.get_acc_and_auc(X = X_train, y = y_train)
print("Accuracy: " , acc_train, "\nAUC: ",auc_train)

acc_train, auc_train = clf.get_acc_and_auc(X = X_test, y = y_test)
print("Accuracy: " , acc_train, "\nAUC: ",auc_train)

The Risk Score is:
1.    CDKN2A_-2     -2 point(s) |   ...
2.      NLGN1_0      2 point(s) | + ...
3.    FANCD2_-2      5 point(s) | + ...
4.    RPUSD3_-1      4 point(s) | + ...
5.      MIR31_2      5 point(s) | + ...
6.     DCTN4_-1     -3 point(s) | + ...
7.     PLXDC1_1     -2 point(s) | + ...
8.     PCDH20_1      2 point(s) | + ...
9.    ZNF287_-1     -2 point(s) | + ...
10.        SI_-1     -2 point(s) | + ...
                          SCORE | =    
SCORE |  -11.0  |  -9.0  |  -8.0  |  -7.0  |  -6.0  |  -5.0  |  -4.0  |  -3.0  |  -2.0  |  -1.0  |   0.0  |   1.0  |   2.0  |   3.0  |
RISK  |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.0% |   0.1% |   0.2% |   0.6% |   1.6% |   4.4% |  11.4% |
SCORE |   4.0  |   5.0  |   6.0  |   7.0  |   8.0  |   9.0  |  10.0  |  11.0  |  12.0  |  13.0  |  14.0  |  15.0  |  16.0  |  18.0  |
RISK  |  26.4% |  50.0% |  73.6% |  88.6% |  95.6% |  98.4% |  99.4% |  99.8% |  99.9% | 100.0% | 100.0% | 100.0% | 100.0% | 100.0% |
Tra